<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Заполнение-пропусков" data-toc-modified-id="Заполнение-пропусков-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Заполнение пропусков</a></span></li><li><span><a href="#Проверка-на-дубликаты" data-toc-modified-id="Проверка-на-дубликаты-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Проверка на дубликаты</a></span></li><li><span><a href="#Удаление-лишних-признаков" data-toc-modified-id="Удаление-лишних-признаков-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Удаление лишних признаков</a></span></li><li><span><a href="#Преобразование-данных-для--обучения-модели" data-toc-modified-id="Преобразование-данных-для--обучения-модели-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Преобразование данных для  обучения модели</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Разбиение-данных-на-выборки" data-toc-modified-id="Разбиение-данных-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разбиение данных на выборки</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

**Цель исследования** — построение предсказательной модели по оттоку клиентов по по различным признакам с значением *F1*-меры более 0.59.

**Ход исследования**

Данные о клиентах находятся в файле `Churn.csv`.

На этапе анализа данных необходимо будет разделить выборку на обучающую, валидационную и тестовую выборки, исследовать исследовать баланс классов, обучить модель без учета дисбаланса, а затем, учитывая дисбаланс, улучшить качество модели. Обучить разные модели и определить лучшую. Провести финальное тестирование.
 
Таким образом, исследование пройдёт в четыре этапа:
 1. Подготовка данных;
 2. Исследование задачи;
 3. Борьба с дисбалансом;
 4. Тестирование модели.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.utils import shuffle
import warnings

warnings.filterwarnings('ignore')

In [2]:
clients = pd.read_csv('Churn.csv')
display(clients.head(10))
display(clients.info())
display(clients.describe())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Таблица **clients**:

**Признаки**
* `RowNumber` — индекс строки в данных
* `CustomerId` — уникальный идентификатор клиента
* `Surname` — фамилия
* `CreditScore` — кредитный рейтинг
* `Geography` — страна проживания
* `Gender` — пол
* `Age` — возраст
* `Tenure` — сколько лет человек является клиентом банка
* `Balance` — баланс на счёте
* `NumOfProducts` — количество продуктов банка, используемых клиентом
* `HasCrCard` — наличие кредитной карты
* `IsActiveMember` — активность клиента
* `EstimatedSalary` — предполагаемая зарплата

**Целевой признак**
* `Exited` — факт ухода клиента

### Заполнение пропусков

В результате обзора данных обнаружены пропуски в признаке `Tenure`.

In [3]:
clients['Tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

Этот признак можно было бы заполнить медианным значением, нулём или просто удалить строки с пропусками, так как их сравнительно немного (около 9-10%). Рассматривая распределение клинтов по годам, можно заметить аномально маленькое количество свежих клиентов (0 лет), хотя логика подсказывает, что таких должно быть довольно много. На основе предположения, что пропуски связаны с новыми клиентами, заполним пропуски нулём

In [4]:
clients = clients.fillna(0)

In [5]:
display(clients.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

### Проверка на дубликаты

Проведём проверку на дубликаты по `CustomerId`

In [6]:
clients['CustomerId'].duplicated().sum()

0

Дубликатов по id необнаружено.

### Удаление лишних признаков

В результате обзора данных сразу видны признаки, которые можно отбросить, как явно не влияющие на целевой признак. Это признаки `RowNumber`, `CustomerId` и `Surname`.

In [7]:
clients = clients.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

### Преобразование данных для  обучения модели

Кроме того нужно провести прямое кодирование для категориальных признаков (`Geography`, `Gender`) и масштабирование для количественных (`CreditScore`, `Age`, `Balance`, `EstimatedSalary`). Масштабирование проведём уже после разбиения данных на выборки.

In [8]:
clients = pd.get_dummies(clients, columns = ['Geography', 'Gender'], drop_first = True)

**Вывод**

Произведён обзор данных, заполнены пропуски. Данные проверены на дубликаты. Отброшены лишние для нашего исследования признаки и проведено прямое кодирование для категориальных признаков.

## Исследование задачи

### Разбиение данных на выборки

Дважды применим функцию train_test_split, чтобы получить тренировочную, валидационную и тестовую выборку в соотношении 0.6:0.2:0.2

In [9]:
clients_train, clients_q = train_test_split(clients, test_size=0.4, random_state=8)
clients_valid, clients_test = train_test_split(clients_q, test_size=0.5, random_state=8)
print('Величина тренировочной выборки:', clients_train.shape[0])
print('Величина валидационной выборки:', clients_valid.shape[0])
print('Величина тестовой выборки:', clients_test.shape[0])

Величина тренировочной выборки: 6000
Величина валидационной выборки: 2000
Величина тестовой выборки: 2000


Разделим данные на признаки и целевые признаки

In [10]:
train_features = clients_train.drop('Exited', axis = 1)
train_target = clients_train['Exited']

valid_features = clients_valid.drop('Exited', axis = 1)
valid_target = clients_valid['Exited']

test_features = clients_test.drop('Exited', axis = 1)
test_target = clients_test['Exited']

Проведём масштабирование количественных признаков

In [11]:
numeric = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(train_features[numeric])

train_features[numeric] = scaler.transform(train_features[numeric])
valid_features[numeric] = scaler.transform(valid_features[numeric])
test_features[numeric] = scaler.transform(test_features[numeric]);

Рассмотрим баланс классов

In [12]:
print(train_target.value_counts())

0    4765
1    1235
Name: Exited, dtype: int64


Классы несбалансированы. Сначала обучим модели без учета дисбаланса.

Напишем функцию, которая будет создавать сводные таблицы с метриками для разных моделей.

In [13]:
def model_metrics(model, rec):
    if rec == 'test':
        probabilities = model.predict_proba(test_features)[:,1]
        prediction = model.predict(test_features)
        d = {'Accuracy': [model.score(test_features, test_target)],
                 'Precision': [precision_score(test_target, prediction)],
                 'Recall': [recall_score(test_target, prediction)],
                 'F1': [f1_score(test_target, prediction)],
                 'AUC_ROC': [roc_auc_score(test_target, probabilities)]}
        return pd.DataFrame(data = d)
    elif rec == 'valid':
        probabilities = model.predict_proba(valid_features)[:,1]
        prediction = model.predict(valid_features)
        d = {'Accuracy': [model.score(valid_features, valid_target)],
                 'Precision': [precision_score(valid_target, prediction)],
                 'Recall': [recall_score(valid_target, prediction)],
                 'F1': [f1_score(valid_target, prediction)],
                 'AUC_ROC': [roc_auc_score(valid_target, probabilities)]}
        return pd.DataFrame(data = d)

Опредилим гиперпараметры для методов решающего дерева и случайного леса без учета дисбаланса классов

In [14]:
best_f1 = 0
best_max_depth = 0
for i in range(1, 51):
    d_tree_model = DecisionTreeClassifier(max_depth = i, random_state = 8)
    d_tree_model.fit(train_features, train_target)
    prediction_valid = d_tree_model.predict(valid_features)
    f1 = f1_score(valid_target, prediction_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_max_depth = i
print(f'Максимальное значение f1 для модели DecisionTree на валидационной выборке без учета дисбаланса классов = {best_f1} с max_depth = {best_max_depth}')

Максимальное значение f1 для модели DecisionTree на валидационной выборке без учета дисбаланса классов = 0.5342465753424658 с max_depth = 10


In [15]:
best_est = 0
best_depth = 0
best_f1 = 0
for est in range(1, 51):
    for depth in range (1, 15):
        R_forest_model = RandomForestClassifier(max_depth = depth, n_estimators = est, random_state = 8)
        R_forest_model.fit(train_features, train_target)
        predictions_valid = R_forest_model.predict(valid_features)
        f1 = f1_score(valid_target, predictions_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
print(f'Максимальное значение f1 для модели RandomForest на валидационной выборке без учета дисбаланса классов = {best_f1} с max_depth = {best_depth} и n_estimators = {best_est}')

Максимальное значение f1 для модели RandomForest на валидационной выборке без учета дисбаланса классов = 0.5816485225505444 с max_depth = 14 и n_estimators = 28


Обучим модели без учета дисбаланса классов

In [16]:
tree_model = DecisionTreeClassifier(random_state = 8, max_depth = 10)
tree_model.fit(train_features, train_target)

forest_model = RandomForestClassifier(random_state = 8, n_estimators = 28, max_depth = 14)
forest_model.fit(train_features, train_target)

metrics_dtc_d = model_metrics(tree_model, 'valid')
metrics_dtc_d['balance_method'] = 'disbalanced'
metrics_dtc_d['name'] = 'decision tree'

metrics_rfc_d = model_metrics(forest_model, 'valid')
metrics_rfc_d['balance_method'] = 'disbalanced'
metrics_rfc_d['name'] = 'random forest'

dis_metrics = pd.concat([metrics_dtc_d, metrics_rfc_d])
dis_metrics

,Accuracy,Precision,Recall,F1,AUC_ROC,balance_method,name
0,0.8300,0.578635,0.496183,0.534247,0.753944,disbalanced,decision tree
0,0.8655,0.748000,0.475827,0.581649,0.831840,disbalanced,random forest


**Вывод**

Были обучены разные модели без учёта дисбаланса классов. 
В тренировочной выборке виден явный дисбаланс в сторону 0. Частоты появления 0 и 1 относятся как 4:1.
Даже не учитывая дисбаланс классов была получена модель с довольно высоким значением f1, которая практически удовлетворяет задаче. Это модель случайного леса. Для неё f1 = 0.582 и auc_roc = 0.832.
На втором месте модель решающего дерева с f1 = 0.534 и auc_roc = 0.754.
Хуже всего справляется модель логистической регрессии с f1 = 0.337, но при это имеет также довольно высокое auc_roc = 0.784.
При этом все модели имеют не очень большое значение полноты за счёт дисбаланса классов в тренировочной выборке.
Все модели имеют довольно высокое значение площади под кривой ошибок, если сравнивать с случайной моделью (0.5).
Для увеличения полноты необходимо применить методы борьбы с дисбалансом.

## Борьба с дисбалансом

Борьба с дисбалансом может проводится с помощью трёх техник: взвешивания классов, upsampling и downsampling.

Опредилим гиперпараметры для методов решающего дерева и случайного леса для техники взвешивания классов

In [17]:
best_f1 = 0
best_max_depth = 0
for i in range(1, 51):
    d_tree_model = DecisionTreeClassifier(max_depth = i, random_state = 8, class_weight = 'balanced')
    d_tree_model.fit(train_features, train_target)
    prediction_valid = d_tree_model.predict(valid_features)
    f1 = f1_score(valid_target, prediction_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_max_depth = i
print(f'Максимальное значение f1 для модели DecisionTree на валидационной выборке с техникой взвешивания классов = {best_f1} с max_depth = {best_max_depth}')

Максимальное значение f1 для модели DecisionTree на валидационной выборке с техникой взвешивания классов = 0.5772972972972974 с max_depth = 8


In [18]:
best_est = 0
best_depth = 0
best_f1 = 0
for est in range(1, 51):
    for depth in range (1, 15):
        R_forest_model = RandomForestClassifier(max_depth = depth, n_estimators = est, random_state = 8, class_weight = 'balanced')
        R_forest_model.fit(train_features, train_target)
        predictions_valid = R_forest_model.predict(valid_features)
        f1 = f1_score(valid_target, predictions_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
print(f'Максимальное значение f1 для модели RandomForest на валидационной выборке с техникой взвешивания классов = {best_f1} с max_depth = {best_depth} и n_estimators = {best_est}')

Максимальное значение f1 для модели RandomForest на валидационной выборке с техникой взвешивания классов = 0.6218274111675127 с max_depth = 10 и n_estimators = 34


In [19]:
tree_model = DecisionTreeClassifier(random_state = 8, max_depth = 8, class_weight = 'balanced')
tree_model.fit(train_features, train_target)

forest_model = RandomForestClassifier(random_state = 8, n_estimators = 34, max_depth = 10, class_weight = 'balanced')
forest_model.fit(train_features, train_target)

metrics_dtc_w = model_metrics(tree_model, 'valid')
metrics_dtc_w['balance_method'] = 'class_weight'
metrics_dtc_w['name'] = 'decision tree'

metrics_rfc_w = model_metrics(forest_model, 'valid')
metrics_rfc_w['balance_method'] = 'class_weight'
metrics_rfc_w['name'] = 'random forest'

dis_metrics = pd.concat([dis_metrics, metrics_dtc_w, metrics_rfc_w])
dis_metrics= dis_metrics.sort_values(by='name')
dis_metrics

,Accuracy,Precision,Recall,F1,AUC_ROC,balance_method,name
0,0.8300,0.578635,0.496183,0.534247,0.753944,disbalanced,decision tree
0,0.8045,0.501880,0.679389,0.577297,0.803672,class_weight,decision tree
0,0.8655,0.748000,0.475827,0.581649,0.831840,disbalanced,random forest
0,0.8510,0.620253,0.623410,0.621827,0.844747,class_weight,random forest


Напишем функцию, которая будет проводить увеличение выборки

In [20]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=8)
    
    return features_upsampled, target_upsampled

In [21]:
features_upsampled, target_upsampled = upsample(train_features, train_target, 4)

Определим гиперпараметры для увеличенной выборки

In [22]:
best_f1 = 0
best_max_depth = 0
for i in range(1, 51):
    d_tree_model = DecisionTreeClassifier(max_depth = i, random_state = 8)
    d_tree_model.fit(features_upsampled, target_upsampled)
    prediction_valid = d_tree_model.predict(valid_features)
    f1 = f1_score(valid_target, prediction_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_max_depth = i
print(f'Максимальное значение f1 для модели DecisionTree на валидационной выборке с техникой увеличения выборки = {best_f1} с max_depth = {best_max_depth}')

Максимальное значение f1 для модели DecisionTree на валидационной выборке с техникой увеличения выборки = 0.5714285714285714 с max_depth = 8


In [23]:
best_est = 0
best_depth = 0
best_f1 = 0
for est in range(1, 51):
    for depth in range (1, 15):
        R_forest_model = RandomForestClassifier(max_depth = depth, n_estimators = est, random_state = 8)
        R_forest_model.fit(features_upsampled, target_upsampled)
        predictions_valid = R_forest_model.predict(valid_features)
        f1 = f1_score(valid_target, predictions_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
print(f'Максимальное значение f1 для модели RandomForest на валидационной выборке с техникой увеличения выборки = {best_f1} с max_depth = {best_depth} и n_estimators = {best_est}')

Максимальное значение f1 для модели RandomForest на валидационной выборке с техникой увеличения выборки = 0.6056701030927835 с max_depth = 14 и n_estimators = 33


In [24]:
tree_model = DecisionTreeClassifier(random_state = 8, max_depth = 8)
tree_model.fit(features_upsampled, target_upsampled)

forest_model = RandomForestClassifier(random_state = 8, n_estimators = 33, max_depth = 14)
forest_model.fit(features_upsampled, target_upsampled)

metrics_dtc_u = model_metrics(tree_model, 'valid')
metrics_dtc_u['balance_method'] = 'upsampling'
metrics_dtc_u['name'] = 'decision tree'

metrics_rfc_u = model_metrics(forest_model, 'valid')
metrics_rfc_u['balance_method'] = 'upsampling'
metrics_rfc_u['name'] = 'random forest'

dis_metrics = pd.concat([dis_metrics, metrics_dtc_u, metrics_rfc_u])
dis_metrics= dis_metrics.sort_values(by='name')
dis_metrics

,Accuracy,Precision,Recall,F1,AUC_ROC,balance_method,name
0,0.8300,0.578635,0.496183,0.534247,0.753944,disbalanced,decision tree
0,0.8045,0.501880,0.679389,0.577297,0.803672,class_weight,decision tree
0,0.7900,0.477002,0.712468,0.571429,0.805042,upsampling,decision tree
0,0.8655,0.748000,0.475827,0.581649,0.831840,disbalanced,random forest
0,0.8510,0.620253,0.623410,0.621827,0.844747,class_weight,random forest
0,0.8470,0.613577,0.597964,0.605670,0.841181,upsampling,random forest


Напишем функцию, которая будет проводить уменьшение выборки

In [25]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=8)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=8)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=8)
    
    return features_downsampled, target_downsampled

In [26]:
features_downsampled, target_downsampled = downsample(train_features, train_target, 0.25)

Определим гиперпараметры для уменьшенной выборки

In [27]:
best_f1 = 0
best_max_depth = 0
for i in range(1, 51):
    d_tree_model = DecisionTreeClassifier(max_depth = i, random_state = 8)
    d_tree_model.fit(features_downsampled, target_downsampled)
    prediction_valid = d_tree_model.predict(valid_features)
    f1 = f1_score(valid_target, prediction_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_max_depth = i
print(f'Максимальное значение f1 для модели DecisionTree на валидационной выборке с техникой уменьшения выборки = {best_f1} с max_depth = {best_max_depth}')

Максимальное значение f1 для модели DecisionTree на валидационной выборке с техникой уменьшения выборки = 0.549950544015826 с max_depth = 6


In [28]:
best_est = 0
best_depth = 0
best_f1 = 0
for est in range(1, 51):
    for depth in range (1, 15):
        R_forest_model = RandomForestClassifier(max_depth = depth, n_estimators = est, random_state = 8)
        R_forest_model.fit(features_downsampled, target_downsampled)
        predictions_valid = R_forest_model.predict(valid_features)
        f1 = f1_score(valid_target, predictions_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
print(f'Максимальное значение f1 для модели RandomForest на валидационной выборке с техникой уменьшения выборки = {best_f1} с max_depth = {best_depth} и n_estimators = {best_est}')

Максимальное значение f1 для модели RandomForest на валидационной выборке с техникой уменьшения выборки = 0.585838991270611 с max_depth = 5 и n_estimators = 43


In [29]:
tree_model = DecisionTreeClassifier(random_state = 8, max_depth = 6)
tree_model.fit(features_downsampled, target_downsampled)

forest_model = RandomForestClassifier(random_state = 8, n_estimators = 43, max_depth = 5)
forest_model.fit(features_downsampled, target_downsampled)

metrics_dtc_d = model_metrics(tree_model, 'valid')
metrics_dtc_d['balance_method'] = 'downsampling'
metrics_dtc_d['name'] = 'decision tree'

metrics_rfc_d = model_metrics(forest_model, 'valid')
metrics_rfc_d['balance_method'] = 'downsampling'
metrics_rfc_d['name'] = 'random forest'

dis_metrics = pd.concat([dis_metrics, metrics_dtc_d, metrics_rfc_d])
dis_metrics= dis_metrics.sort_values(by='name')
dis_metrics

,Accuracy,Precision,Recall,F1,AUC_ROC,balance_method,name
0,0.8300,0.578635,0.496183,0.534247,0.753944,disbalanced,decision tree
0,0.8045,0.501880,0.679389,0.577297,0.803672,class_weight,decision tree
0,0.7900,0.477002,0.712468,0.571429,0.805042,upsampling,decision tree
0,0.7725,0.449838,0.707379,0.549951,0.820844,downsampling,decision tree
0,0.8655,0.748000,0.475827,0.581649,0.831840,disbalanced,random forest
0,0.8510,0.620253,0.623410,0.621827,0.844747,class_weight,random forest
0,0.8470,0.613577,0.597964,0.605670,0.841181,upsampling,random forest
0,0.7865,0.473354,0.768448,0.585839,0.852931,downsampling,random forest


**Вывод**

Были обучены разные модели с применением различных техник борьбы с дисбалансом.
Судя по метрикам f1 и auc_roc качество моделей выросло. Как мы видим по таблице точность немного упала, зато полнота выросла, поэтому f1 и вырос.
Самые качественные модели (по метрике f1) - это модель случайного леса с применением техники взвешивания классов и техники увеличения выборки. Эти модели преодолели требуемый порог в f1 = 0.59. Именно эти модели и будем тестировать в 4 пункте.

## Тестирование модели

In [30]:
forest_model_up = RandomForestClassifier(random_state = 8, n_estimators = 33, max_depth = 14)
forest_model_up.fit(features_upsampled, target_upsampled)

forest_model_w = RandomForestClassifier(random_state = 8, n_estimators = 34, max_depth = 10, class_weight = 'balanced')
forest_model_w.fit(train_features, train_target)

metrics_rfc_up = model_metrics(forest_model_up, 'test')
metrics_rfc_up['balance_method'] = 'upsampling'
metrics_rfc_up['name'] = 'random forest'

metrics_rfc_w = model_metrics(forest_model_w, 'test')
metrics_rfc_w['balance_method'] = 'class_weight'
metrics_rfc_w['name'] = 'random forest'

dis_metrics = pd.concat([metrics_rfc_up, metrics_rfc_w])
dis_metrics= dis_metrics.sort_values(by='name')
dis_metrics

,Accuracy,Precision,Recall,F1,AUC_ROC,balance_method,name
0,0.8475,0.631313,0.611247,0.621118,0.855137,upsampling,random forest
0,0.8520,0.645244,0.613692,0.629073,0.853734,class_weight,random forest


**Вывод**

Обе модели на тестовой выборке показали довольно высокие результаты с F1-мерой больше 0,59. Метрика AUC_ROC также получилась довольно высокой, значительно больше случайной модели (0,5)

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*